In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bookcrossing-dataset/Books Data with Category Language and Summary/Preprocessed_data.csv
/kaggle/input/bookcrossing-dataset/Book reviews/Book reviews/BX_Books.csv
/kaggle/input/bookcrossing-dataset/Book reviews/Book reviews/BX-Book-Ratings.csv
/kaggle/input/bookcrossing-dataset/Book reviews/Book reviews/BX-Users.csv


In [2]:
df = pd.read_csv('/kaggle/input/bookcrossing-dataset/Books Data with Category Language and Summary/Preprocessed_data.csv')
df.drop(['Unnamed: 0','img_s','img_m','img_l'], axis=1, inplace=True)
df.head()

,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,Summary,Language,Category,city,state,country
0,2,"stockton, california, usa",18.0000,0195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,Provides an introduction to classical myths pl...,en,['Social Science'],stockton,california,usa
1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada
2,11400,"ottawa, ontario, canada",49.0000,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,ontario,canada
3,11676,"n/a, n/a, n/a",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],NaN,NaN,NaN
4,41385,"sudbury, ontario, canada",34.7439,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],sudbury,ontario,canada


In [3]:
df.drop_duplicates(subset=['user_id','book_title'],inplace=True)
df.dropna(inplace=True)
df.shape

(978141, 15)

In [4]:
counts = df['user_id'].value_counts()
keep = counts[counts > 10].index
keep

Int64Index([198711,  98391, 153662,  35859, 212898, 278418,  76352, 110973,
            235105,  16795,
            ...
            240033,  91685,  26030,  51511, 148121, 221967, 244892, 223375,
            238527, 246268],
           dtype='int64', length=10234)

In [5]:
df = df[df['user_id'].isin(keep)]
df.shape

(817015, 15)

# Start with a user-based and item-based collab filtering model

In [6]:
counts = df['book_title'].value_counts()
keep = counts[counts > 20].index
keep

Index(['Wild Animus', 'The Lovely Bones: A Novel', 'The Da Vinci Code',
       'The Nanny Diaries: A Novel', 'Bridget Jones's Diary',
       'The Secret Life of Bees',
       'Divine Secrets of the Ya-Ya Sisterhood: A Novel', 'A Painted House',
       'Angels & Demons', 'Life of Pi',
       ...
       'Together', 'The Little Country',
       'The Joy Luck Club (Vintage Contemporaries)', 'The Outsider',
       'LONESOME DOVE  M', 'The Beet Queen',
       'The Celestine Vision: Living the New Spiritual Awareness', 'The Miko',
       'Black Rainbow', 'One Child'],
      dtype='object', length=5562)

In [7]:
df = df[df['book_title'].isin(keep)]
df.shape

(301720, 15)

In [8]:
pivot_df = df.pivot(index='user_id',columns='book_title',values='rating')

In [9]:
pivot_df

book_title,'Salem's Lot,10 Lb. Penalty,1001 Ways to Be Romantic,101 Dalmatians,"14,000 Things to Be Happy About",16 Lighthouse Road,1984,1st to Die: A Novel,2001: A Space Odyssey,2010: Odyssey Two,...,Zia,Zlata's Diary: A Child's Life in Sarajevo,Zodiac: The Eco-Thriller,Zombies of the Gene Pool,Zoya,"\ Lamb to the Slaughter and Other Stories (Penguin 60s S.)""","\O\"" Is for Outlaw""","\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""",e,stardust
user_id,,,,,,,,,,,,,,,,,,,,,
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
avg_ratings = pivot_df.mean(axis=1)
user_rating_df = pivot_df.sub(avg_ratings,axis=0).fillna(0)
user_rating_df

book_title,'Salem's Lot,10 Lb. Penalty,1001 Ways to Be Romantic,101 Dalmatians,"14,000 Things to Be Happy About",16 Lighthouse Road,1984,1st to Die: A Novel,2001: A Space Odyssey,2010: Odyssey Two,...,Zia,Zlata's Diary: A Child's Life in Sarajevo,Zodiac: The Eco-Thriller,Zombies of the Gene Pool,Zoya,"\ Lamb to the Slaughter and Other Stories (Penguin 60s S.)""","\O\"" Is for Outlaw""","\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""",e,stardust
user_id,,,,,,,,,,,,,,,,,,,,,
8,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
99,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
242,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
243,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
254,0.0,0.0,0.0,0.0,0.0,0.0,7.374046,0.0,0.0,0.0,...,0.0,0.0,-1.625954,0.0,0.0,0.0,0.0,0.0,0.0,-1.625954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278633,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
278637,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
278771,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [11]:
from sklearn.neighbors import KNeighborsRegressor

In [12]:
def predict_rating(user,book):
    X = user_rating_df.drop(book,axis=1)
    y = user_rating_df[book]
    knn = KNeighborsRegressor(metric='cosine',n_neighbors=3)
    knn.fit(X,y)
    return knn.predict([X.iloc[user]])

In [13]:
predict_rating(99,'101 Dalmatians')

array([0.])

# Hybrid Model